In [38]:
from tool import *
import sys

# general options
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

## PO DB 업데이트

In [39]:
filepath = 'D:/Downloads/' + [file for file in os.listdir('D:/Downloads/') if 'poshThru' in file][-1] # 파일이름에 poshThru가 들어가는 파일 중에 가장 최신 파일을 선택함
df = pd.read_excel(filepath, sheet_name='DataSet').iloc[:, :48]

if df['Model'][0].split('.')[0] == 'PH30N':
    vendor = 'Wanlida'
else:
    vendor = df['Model'].apply(lambda x:x.split('-')[0]).replace(srt_model).replace(vendor_find)[0]

if vendor not in ['Quanta', 'Pegatron', 'Wanlida', 'Wingtech']:
    sys.exit()

df = df[['Model', 'PO No.', 'Ship To', 'Shipping', 'Cancel', 'PO', 'OQC Report', 'OQC Date', 'OQC Result', 'Ship', 'Issued Date', 'RSD','Ship Date', 'BL No', 'BL Status', 'Method', 'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.', 'Final Destination']]
df.loc[df['PO1 No'].notnull(), 'PO1 No'] = df.loc[df['PO1 No'].notnull(),'PO1 No'].astype('int').astype('str')
df.loc[df['SO No.'].notnull(), 'SO No.'] = df.loc[df['SO No.'].notnull(), 'SO No.'].astype('int').astype('str')
df[['OQC Date', 'PO1 No', 'SO No.', 'Final Destination']] = df[['OQC Date', 'PO1 No', 'SO No.', 'Final Destination']].fillna('-')

df['Ship'].fillna(0, inplace=True)
df['Ship'] = df['Ship'].astype('int')
df['Issued Date'] = pd.to_datetime(df['Issued Date'])
df['PO Week'] = df['Issued Date'].dt.isocalendar().week.apply(lambda x:'{0:02d}'.format(x))
df['PO Month'] = df['Issued Date'].apply(get_month_from_date)
df['PO Year'] = df['Issued Date'].dt.isocalendar().year
df['RSD'] = pd.to_datetime(df['RSD']) # PO의 RSD를 Datetime 형식으로 변경
df['RSD Week'] = df['RSD'].dt.isocalendar().week.apply(lambda x:'{0:02d}'.format(x))
df['RSD Month'] = df['RSD'].apply(get_month_from_date)
df['RSD Year'] = df['RSD'].dt.isocalendar().year
df['RSD Week Year'] = df['RSD'].apply(getfirstdate_year)
df['RSD Week Month'] = df['RSD'].apply(getfirstdate_month)
df['RSD Week Day'] = df['RSD'].apply(getfirstdate_day)
df['RSD Week Name'] = df['RSD Week Year'].astype(str) + '-' + df['RSD Week Month'].astype(str) + '-' + df['RSD Week Day'].astype(str) + '(W' + df['RSD Week'].astype(str) +')'
df.rename(columns={'Model':'Mapping Model.Suffix'}, inplace=True)

if vendor != 'Wanlida':
    df['Model'] = df['Mapping Model.Suffix'].apply(lambda x: x.split('-')[0])
    df['Series'] = df['Model'].replace(srt_model)

df['Ship To'] = df['Ship To'].replace(site_name_adjust_map)
df['Country'] = df['Ship To'].replace(site_map)
df['Region'] = df['Country'].replace(country_map)

if vendor != 'Wanlida':
    df1 = df.groupby(['PO No.', 'Cancel', 'Issued Date', 'PO Year', 'PO Month', 'PO Week', 'Region', 'Country', 'Series', 'Model',
                        'Mapping Model.Suffix', 'Ship To', 'RSD', 'RSD Year', 'RSD Month', 'RSD Week', 'RSD Week Name', 'Method',
                        'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.',
                        'Final Destination'])[['PO']].sum().reset_index().set_index('PO No.')
else:
    df1 = df.groupby(['PO No.', 'Cancel', 'Issued Date', 'PO Year', 'PO Month', 'PO Week', 'Region', 'Country',
                    'Mapping Model.Suffix', 'Ship To', 'RSD', 'RSD Year', 'RSD Month', 'RSD Week', 'RSD Week Name', 'Method',
                    'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.',
                    'Final Destination'])[['PO']].sum().reset_index().set_index('PO No.')  

with open(f'D:/Data/{vendor} PO DB.bin', 'rb') as f:
    PO_df = pickle.load(f)

In [40]:
PO_df1 = PO_df.copy()

In [41]:
prev_po = PO_df.shape[0]

In [42]:
for i in df1.index:
    PO_df.loc[i, :] = df1.loc[i, :]

In [43]:
updated_po = PO_df.shape[0]

In [44]:
with open(f'D:/Data/{vendor} PO DB.bin', 'wb') as f:
    pickle.dump(PO_df, f)
# DB backup 폴더에도 저장
with open(f'D:/Data/DB backup/{vendor} PO DB.bin', 'wb') as f:
    pickle.dump(PO_df, f)

## Shipment result DB 업데이트

In [45]:
with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
     SR_df = pickle.load(f)

df = df[df['Ship Date'].notnull()]
df.loc[:, 'Ship Date'] = pd.to_datetime(df.loc[:, 'Ship Date'])
df['Ship Week'] = df['Ship Date'].dt.isocalendar().week.apply(lambda x:'{0:02d}'.format(x))
df['Ship Month'] = df['Ship Date'].apply(get_month_from_date)
df['Ship Year'] = df['Ship Date'].dt.isocalendar().year
df['Ship Week month'] = df['Ship Date'].apply(getfirstdate_month)
df['Ship Week day'] = df['Ship Date'].apply(getfirstdate_day)
df['Ship Week year'] = df['Ship Date'].apply(getfirstdate_year)
df['Ship Week year'] = df['Ship Week year'].astype(str)
df['Ship Week month'] = df['Ship Week month'].astype(str)
df['Ship Week day'] = df['Ship Week day'].astype(str)
df['Ship Week'] = df['Ship Week'].astype(str)
df['Week Name'] = df['Ship Week year'] + '-' + df['Ship Week month'] + '-' + df['Ship Week day'] + '(W' + df['Ship Week'] + ')'
df = df.set_index(['PO No.', 'BL No'])

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_15052\37446698.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'Ship Date'] = pd.to_datetime(df.loc[:, 'Ship Date'])


In [46]:
SR_df1 = SR_df.copy()

In [47]:
prev_sr = SR_df.shape[0]

In [48]:
for i in df.index:
    SR_df.loc[i, :] = df.loc[i, :]

In [49]:
updated_sr = SR_df.shape[0]

In [50]:
# PO 대비 선적수량이 더 많은 경우가 없는지 check
with open(f'D:/Data/{vendor} PO DB.bin', 'rb') as f:
    PO = pickle.load(f)
with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
    SR = pickle.load(f)

PO = PO[PO['Cancel']=='N']
df = pd.concat([PO['PO'], SR.groupby('PO No.')['Ship'].sum()], axis=1)
df.fillna(0, inplace=True)
df['Open'] = df['PO'] - df['Ship']
df = df[df['Open'] < 0]

In [51]:
if df.shape[0] != 0:
    print(df)
    sys.exit()

In [52]:
with open(f'D:/Data/{vendor} shipment result DB.bin', 'wb') as f:
    pickle.dump(SR_df, f)

In [53]:
with open(f'D:/Data/DB backup/{vendor} shipment result DB.bin', 'wb') as f:
    pickle.dump(SR_df, f)
print(f"{vendor}의 DB를 업데이트 하였습니다. PO는 {updated_po - prev_po} 건, 선적은 {updated_sr - prev_sr} 건 추가 되었습니다. ")
print(filepath)

Quanta의 DB를 업데이트 하였습니다. PO는 0 건, 선적은 14 건 추가 되었습니다. 
D:/Downloads/poshThru_20230428082825.xls


In [54]:
PO_df.drop(index=PO_df1.index).pivot_table('PO', index=['PO No.', 'Mapping Model.Suffix',  'Country'], columns='RSD Week Name', aggfunc=sum) # 추가된 PO 리스트

,,RSD Week Name
PO No.,Mapping Model.Suffix,Country


In [55]:
SR_df.drop(index=SR_df1.index).pivot_table('Ship', index=['Country', 'BL No', 'Mapping Model.Suffix', 'PO No.'], columns=['Week Name', 'Ship Date'], aggfunc='sum').fillna(0) # 추가된 선적실적

Week Name                                            2023-04-24(W17)  \
Ship Date                                                 2023-04-24   
Country BL No      Mapping Model.Suffix PO No.                         
Germany UTOC500667 16T90R-G.AA78G       JPCZ23002544             0.0   
Korea   LGE04008   14T90R-G.AA50K       ZPCZ23001445            33.0   
                   15U50Q-G.AP5PL       ZPCZ23002256           112.0   
                   15U50Q-G.AP70ML      ZPCZ23002255            48.0   
                   15U50Q-S.AP70ML      ZPCZ23001857            30.0   
                                        ZPCZ23002257            32.0   
                   15U50Q-S.AR5CK       ZPCZ23001867            36.0   
                   15U50Q-S.AS7DL       ZPCZ23002460            48.0   
                   15U50Q-S.AS7GL       ZPCZ23002264           112.0   
                   15UD50Q-G.AX3DK      ZPCZ23002465            36.0   
                   15UD50Q-G.AX50K      ZPCZ23001853           160.0   
                                        ZPCZ23002060            80.0   
                                        ZPCZ23002254            30.0   
        LGE04011   14T90R-G.AP56ML      ZPCZ23002468             0.0   

Week Name                                                        
Ship Date                                            2023-04-27  
Country BL No      Mapping Model.Suffix PO No.                   
Germany UTOC500667 16T90R-G.AA78G       JPCZ23002544       35.0  
Korea   LGE04008   14T90R-G.AA50K       ZPCZ23001445        0.0  
                   15U50Q-G.AP5PL       ZPCZ23002256        0.0  
                   15U50Q-G.AP70ML      ZPCZ23002255        0.0  
                   15U50Q-S.AP70ML      ZPCZ23001857        0.0  
                                        ZPCZ23002257        0.0  
                   15U50Q-S.AR5CK       ZPCZ23001867        0.0  
                   15U50Q-S.AS7DL       ZPCZ23002460        0.0  
                   15U50Q-S.AS7GL       ZPCZ23002264        0.0  
                   15UD50Q-G.AX3DK      ZPCZ23002465        0.0  
                   15UD50Q-G.AX50K      ZPCZ23001853        0.0  
                                        ZPCZ23002060        0.0  
                                        ZPCZ23002254        0.0  
        LGE04011   14T90R-G.AP56ML      ZPCZ23002468      130.0

In [56]:
get_shipment_result('Quanta', 0)

Ship Date                                                         2023-04-24  \
Country    BL No        BL Status    Series Mapping Model.Suffix               
Austrailia UTOC500665   CC Requested 16T90R 16T90R-G.AA78A                 0   
Germany    UTOC500667   CC Requested 16T90R 16T90R-G.AA78G                 0   
Korea      LGE04007     SD Confirmed 15U40Q 15UD40Q-G.AX5DK              800   
                                     15U50Q 15U50Q-G.AP70ML               70   
                                            15U50Q-S.AS7DL               372   
                                            15U50Q-S.AS7GL               200   
                                            15UD50Q-G.AX3DK              150   
                                     16T90R 16T90R-G.AP7WL                 1   
           LGE04008     SD Confirmed 14T90R 14T90R-G.AA50K                33   
                                     15U50Q 15U50Q-G.AP5PL               112   
                                            15U50Q-G.AP70ML               48   
                                            15U50Q-S.AP70ML               62   
                                            15U50Q-S.AR5CK                36   
                                            15U50Q-S.AS7DL                48   
                                            15U50Q-S.AS7GL               112   
                                            15UD50Q-G.AX3DK               36   
                                            15UD50Q-G.AX50K              270   
           LGE04011     SD Confirmed 14T90R 14T90R-G.AP56ML                0   
           PLISH4E20147 SD Confirmed 14T90Q 14T90Q-G.AP79L                 7   
                                     14T90R 14T90R-G.AA5CK                 5   
                                            14T90R-G.AP7WL                10   
                                     15U40Q 15UD40Q-G.AX5DK             1206   
                                     15U40R 15U40R-G.ARFWK                20   
                                            15UD40R-G.AX36K               80   
                                     15U50Q 15U50Q-S.AR5CK                 6   
                                     16T90R 16T90R-G.AA5CK                20   
                                            16T90R-G.AA76K                39   
                                            16T90R-G.AP76ML               10   
                                            16T90R-G.AP7WL                13   
Taiwan     UTOC500666   CC Requested 14T90R 14T90R-G.AA74C2                0   
US         UTOC500664   CC Requested 14T90R 14T90R-K.AAB6U1                0   
                                            14T90R-K.AAB8U1                0   
                                     16T90Q 16T90Q-K.ADG9U1                0   
                                     16T90R 16T90R-K.ADB9U1                0   

Ship Date                                                         2023-04-26  \
Country    BL No        BL Status    Series Mapping Model.Suffix               
Austrailia UTOC500665   CC Requested 16T90R 16T90R-G.AA78A                54   
Germany    UTOC500667   CC Requested 16T90R 16T90R-G.AA78G                 0   
Korea      LGE04007     SD Confirmed 15U40Q 15UD40Q-G.AX5DK                0   
                                     15U50Q 15U50Q-G.AP70ML                0   
                                            15U50Q-S.AS7DL                 0   
                                            15U50Q-S.AS7GL                 0   
                                            15UD50Q-G.AX3DK                0   
                                     16T90R 16T90R-G.AP7WL                 0   
           LGE04008     SD Confirmed 14T90R 14T90R-G.AA50K                 0   
                                     15U50Q 15U50Q-G.AP5PL                 0   
                                            15U50Q-G.AP70ML                0   
                                            15U50Q-S.AP70ML                0   